# 检测器模块

## 数据集处理（转化为CocoVID格式）

### 训练集

In [90]:
!rm ./data/VisDrone_MOT_tiny/train/cocoformat.json

In [91]:
# convert the dataset to coco format
!python ./tools/convert_datasets/visdronemot/visdronemot2coco.py -d ./data/VisDrone_MOT_tiny/train

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 24.59it/s]


### 测试集

In [92]:
!rm ./data/VisDrone_MOT_tiny/test/cocoformat.json

In [93]:
# convert the dataset to coco format
!python ./tools/convert_datasets/visdronemot/visdronemot2coco.py -d ./data/VisDrone_MOT_tiny/test

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 26.26it/s]


## 修改训练配置

In [3]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('./configs/det/faster-rcnn_r50_fpn_4e_visdrone_mot.py')
cfg.data_root = 'data/VisDrone_MOT_tiny/'
cfg.data.test.ann_file = cfg.data.test.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.train.ann_file = cfg.data.train.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.val.ann_file = cfg.data.val.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

cfg.data.test.img_prefix = cfg.data.test.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.train.img_prefix = cfg.data.train.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.val.img_prefix = cfg.data.val.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

cfg.work_dir = './tutorial_exps/detector'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.device = "cuda"
# cfg.gpu_ids = range(1)
cfg.gpu_ids = [1]
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    detector=dict(
        type='FasterRCNN',
        backbone=dict(
            type='ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            style='pytorch',
            init_cfg=dict(
                type='Pretrained', checkpoint='torchvision://resnet50')),
        neck=dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            num_outs=5),
        rpn_head=dict(
            type='RPNHead',
            in_channels=256,
            feat_channels=256,
            anchor_generator=dict(
                type='AnchorGenerator',
                scales=[8],
                ratios=[0.5, 1.0, 2.0],
                strides=[4, 8, 16, 32, 64]),
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0.0,

## 检测器训练&验证

In [4]:
import os.path as osp

from mmtrack.datasets import build_dataset
from mmdet.apis import train_detector as train_model
from mmdet.models import build_detector as build_model

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
model = build_model(cfg.model.detector)
model.init_weights()
datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES
train_model(model, datasets, cfg, validate=True)

2022-11-08 15:32:12,775 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'}
2022-11-08 15:32:12,775 - mmcv - INFO - load model from: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
2022-11-08 15:32:12,776 - mmcv - INFO - load checkpoint from http path: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
2022-11-08 15:32:12,871 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([3, 1024]).
size mism

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2022-11-08 15:32:31,467 - mmdet - INFO - Epoch [1][50/164]	lr: 9.902e-03, eta: 0:03:42, time: 0.367, data_time: 0.047, memory: 2946, loss_rpn_cls: 0.0418, loss_rpn_bbox: 0.0697, loss_cls: 0.5932, acc: 74.4766, loss_bbox: 0.4064, loss: 1.1112
2022-11-08 15:32:47,810 - mmdet - INFO - Epoch [1][100/164]	lr: 1.980e-02, eta: 0:03:12, time: 0.327, data_time: 0.005, memory: 2946, loss_rpn_cls: 0.0235, loss_rpn_bbox: 0.0680, loss_cls: 0.3227, acc: 86.5273, loss_bbox: 0.2488, loss: 0.6630
2022-11-08 15:33:04,338 - mmdet - INFO - Epoch [1][150/164]	lr: 2.000e-02, eta: 0:02:52, time: 0.331, data_time: 0.005, memory: 2946, loss_rpn_cls: 0.0241, loss_rpn_bbox: 0.0575, loss_cls: 0.2914, acc: 87.7500, loss_bbox: 0.2071, loss: 0.5801
2022-11-08 15:33:08,932 - mmdet - INFO - Saving checkpoint at 1 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 118/118, 14.7 task/s, elapsed: 8s, ETA:     0s

2022-11-08 15:33:17,743 - mmdet - INFO - Evaluating bbox...


...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 118/118, 14.5 task/s, elapsed: 8s, ETA:     0s

2022-11-08 15:36:37,725 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


2022-11-08 15:36:38,772 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.287
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.639
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.072
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.354
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.438
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.393
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.393
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.393
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.142
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.463
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=0.94s).
Accumulating evaluation results...
DONE (t=0.09s).


# ReID模块

## 数据集处理（转化为ReID数据集）

In [17]:
!rm -r ./data/VisDrone_MOT_tiny/train/reid 
!rm -r ./data/VisDrone_MOT_tiny/train/reid_meta
!rm -r ./data/VisDrone_MOT_tiny/test/reid 
!rm -r ./data/VisDrone_MOT_tiny/test/reid_meta

rm: cannot remove './data/VisDrone_MOT_tiny/test/reid': No such file or directory
rm: cannot remove './data/VisDrone_MOT_tiny/test/reid_meta': No such file or directory


In [18]:
!python ./tools/convert_datasets/visdronemot/visdronemot2reid.py -d ./data/VisDrone_MOT_tiny/train --min-per-person=4 --max-per-person=10

corp reid images...
100%|█████████████████████████████████████████████| 2/2 [00:04<00:00,  2.23s/it]
generate reid labels...


In [19]:
!python ./tools/convert_datasets/visdronemot/visdronemot2reid.py -d ./data/VisDrone_MOT_tiny/test --min-per-person=4 --max-per-person=10

corp reid images...
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.47s/it]
generate reid labels...


## 修改训练配置

In [106]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('./configs/reid/resnet50_b32x8_VisDroneMOT.py')
cfg.data_root = 'data/VisDrone_MOT_tiny/'
# cfg.data.train.ann_file = 'data/VisDrone_MOT_tiny/train/reid_meta/train.txt'
cfg.data.train.ann_file = cfg.data.train.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.val.ann_file = cfg.data.val.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.test.ann_file = cfg.data.test.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

cfg.data.train.data_prefix = cfg.data.train.data_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.val.data_prefix = cfg.data.val.data_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.test.data_prefix = cfg.data.test.data_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

# learning policy
cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=200,
    warmup_ratio=1.0 / 200,
    step=[1])
cfg.total_epochs = 2

cfg.work_dir = './tutorial_exps/reid'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.device = "cuda"
# cfg.gpu_ids = range(1)
cfg.gpu_ids = [0]
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'ReIDDataset'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadMultiImagesFromFile', to_float32=True),
    dict(
        type='SeqResize',
        img_scale=(128, 256),
        share_params=False,
        keep_ratio=False,
        bbox_clip_border=False,
        override=False),
    dict(
        type='SeqRandomFlip',
        share_params=False,
        flip_ratio=0.5,
        direction='horizontal'),
    dict(
        type='SeqNormalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='VideoCollect', keys=['img', 'gt_label']),
    dict(type='ReIDFormatBundle')
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', img_scale=(128, 256), keep_ratio=False),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),

## ReID模块训练&验证

In [29]:
from mmtrack.datasets import build_dataset
from mmdet.apis import train_detector as train_model
from mmtrack.models import build_reid as build_model


model = build_model(cfg.model.reid)
model.init_weights()
datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES

train_model(model, datasets, cfg, validate=True)

2022-11-08 03:12:05,701 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth'}
2022-11-08 03:12:05,702 - mmcv - INFO - load model from: https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth
2022-11-08 03:12:05,702 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth
2022-11-08 03:12:05,747 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.fc.weight, head.fc.bias

missing keys in source state_dict: head.fcs.0.fc.weight, head.fcs.0.fc.bias, head.fcs.0.bn.weight, head.fcs.0.bn.bias, head.fcs.0.bn.running_mean, head.fcs.0.bn.running_var, head.fc_out.weight, head.fc_out.bias, head.bn.weight, head.bn.bias, head.bn.running_mean, head.bn.

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 294/294, 132.5 task/s, elapsed: 2s, ETA:     0s

2022-11-08 03:13:30,394 - mmdet - INFO - Epoch(val) [1][294]	mAP: 0.6870
2022-11-08 03:13:39,146 - mmdet - INFO - Epoch [2][50/604]	lr: 1.000e-02, eta: 0:01:15, time: 0.174, data_time: 0.045, memory: 2587, triplet_loss: 0.2025, ce_loss: 1.8454, top-1: 41.8750, loss: 2.0479
2022-11-08 03:13:45,746 - mmdet - INFO - Epoch [2][100/604]	lr: 1.000e-02, eta: 0:01:08, time: 0.132, data_time: 0.003, memory: 2587, triplet_loss: 0.1795, ce_loss: 1.4846, top-1: 54.3125, loss: 1.6641
2022-11-08 03:13:52,342 - mmdet - INFO - Epoch [2][150/604]	lr: 1.000e-02, eta: 0:01:01, time: 0.132, data_time: 0.003, memory: 2587, triplet_loss: 0.2270, ce_loss: 1.4436, top-1: 50.8750, loss: 1.6706
2022-11-08 03:13:58,934 - mmdet - INFO - Epoch [2][200/604]	lr: 1.000e-02, eta: 0:00:54, time: 0.132, data_time: 0.003, memory: 2587, triplet_loss: 0.1133, ce_loss: 1.3199, top-1: 56.5625, loss: 1.4332
2022-11-08 03:14:05,525 - mmdet - INFO - Epoch [2][250/604]	lr: 1.000e-02, eta: 0:00:48, time: 0.132, data_time: 0.003, 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 294/294, 131.1 task/s, elapsed: 2s, ETA:     0s

2022-11-08 03:14:55,239 - mmdet - INFO - Epoch(val) [2][294]	mAP: 0.7610


# 验证跟踪模型（检测器+跟踪）

In [5]:
import mmcv
from mmdet.apis import set_random_seed
cfg = mmcv.Config.fromfile('./configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_visdrone_mot.py')
cfg.data_root = 'data/VisDrone_MOT_tiny/'
cfg.data.test.ann_file = cfg.data.test.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.train.ann_file = cfg.data.train.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.val.ann_file = cfg.data.val.ann_file.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

cfg.data.test.img_prefix = cfg.data.test.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.train.img_prefix = cfg.data.train.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')
cfg.data.val.img_prefix = cfg.data.val.img_prefix.replace('data/VisDrone_MOT/','data/VisDrone_MOT_tiny/')

cfg.model.detector.init_cfg.checkpoint = './tutorial_exps/detector/epoch_4.pth'
cfg.model.reid.init_cfg.checkpoint = './tutorial_exps/reid/epoch_2.pth'

cfg.work_dir = './tutorial_exps'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.device = 'cuda'
# cfg.gpu_ids = range(1)
cfg.gpu_ids = [0]
cfg.data.test.test_mode = True
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    detector=dict(
        type='FasterRCNN',
        backbone=dict(
            type='ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            style='pytorch',
            init_cfg=dict(
                type='Pretrained', checkpoint='torchvision://resnet50')),
        neck=dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            num_outs=5),
        rpn_head=dict(
            type='RPNHead',
            in_channels=256,
            feat_channels=256,
            anchor_generator=dict(
                type='AnchorGenerator',
                scales=[8],
                ratios=[0.5, 1.0, 2.0],
                strides=[4, 8, 16, 32, 64]),
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0.0,

In [6]:
from mmtrack.datasets import build_dataloader
from mmtrack.apis import init_model
from mmcv.parallel import MMDataParallel
from mmtrack.apis import single_gpu_test
from mmtrack.datasets import build_dataset

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False)

# build the model and load checkpoint
model = init_model(cfg)

model = MMDataParallel(model, device_ids=cfg.gpu_ids)
outputs = single_gpu_test(model, data_loader)

eval_kwargs = cfg.get('evaluation', {}).copy()
# hard-code way to remove EvalHook args
eval_hook_args = [
    'interval', 'tmpdir', 'start', 'gpu_collect', 'save_best',
    'rule', 'by_epoch'
]
for key in eval_hook_args:
    eval_kwargs.pop(key, None)
eval_kwargs.update(dict(metric=['track']))
metric = dataset.evaluate(outputs, **eval_kwargs)
print(metric)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


2022-11-08 15:38:37,002 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/detector/epoch_4.pth'}
2022-11-08 15:38:37,003 - mmcv - INFO - load model from: ./tutorial_exps/detector/epoch_4.pth
2022-11-08 15:38:37,004 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/detector/epoch_4.pth
2022-11-08 15:38:37,195 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/reid/epoch_2.pth'}
2022-11-08 15:38:37,196 - mmcv - INFO - load model from: ./tutorial_exps/reid/epoch_2.pth
2022-11-08 15:38:37,196 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/reid/epoch_2.pth
2022-11-08 15:38:39,118 - mmcv - INFO - 
detector.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,119 - mmcv - INFO - 
detector.backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps

2022-11-08 15:38:39,140 - mmcv - INFO - 
detector.backbone.layer2.0.downsample.1.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,141 - mmcv - INFO - 
detector.backbone.layer2.1.conv1.weight - torch.Size([128, 512, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,142 - mmcv - INFO - 
detector.backbone.layer2.1.bn1.weight - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,143 - mmcv - INFO - 
detector.backbone.layer2.1.bn1.bias - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,143 - mmcv - INFO - 
detector.backbone.layer2.1.conv2.weight - torch.Size([128, 128, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,144 - mmcv - INFO - 
detector.backbone.layer2.1.bn2.weight - torch.Size([128]): 
PretrainedInit: load from ./tutor

2022-11-08 15:38:39,162 - mmcv - INFO - 
detector.backbone.layer3.1.bn3.bias - torch.Size([1024]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,163 - mmcv - INFO - 
detector.backbone.layer3.2.conv1.weight - torch.Size([256, 1024, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,163 - mmcv - INFO - 
detector.backbone.layer3.2.bn1.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,164 - mmcv - INFO - 
detector.backbone.layer3.2.bn1.bias - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,164 - mmcv - INFO - 
detector.backbone.layer3.2.conv2.weight - torch.Size([256, 256, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,169 - mmcv - INFO - 
detector.backbone.layer3.2.bn2.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exp

2022-11-08 15:38:39,192 - mmcv - INFO - 
detector.backbone.layer4.0.downsample.1.bias - torch.Size([2048]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,192 - mmcv - INFO - 
detector.backbone.layer4.1.conv1.weight - torch.Size([512, 2048, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,193 - mmcv - INFO - 
detector.backbone.layer4.1.bn1.weight - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,193 - mmcv - INFO - 
detector.backbone.layer4.1.bn1.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,193 - mmcv - INFO - 
detector.backbone.layer4.1.conv2.weight - torch.Size([512, 512, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,194 - mmcv - INFO - 
detector.backbone.layer4.1.bn2.weight - torch.Size([512]): 
PretrainedInit: load from ./tut

2022-11-08 15:38:39,215 - mmcv - INFO - 
detector.roi_head.bbox_head.shared_fcs.1.bias - torch.Size([1024]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:38:39,216 - mmcv - INFO - 
reid.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,216 - mmcv - INFO - 
reid.backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,217 - mmcv - INFO - 
reid.backbone.bn1.bias - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,217 - mmcv - INFO - 
reid.backbone.layer1.0.conv1.weight - torch.Size([64, 64, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,217 - mmcv - INFO - 
reid.backbone.layer1.0.bn1.weight - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,218 - mmcv - INFO - 

2022-11-08 15:38:39,239 - mmcv - INFO - 
reid.backbone.layer2.1.bn2.bias - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,240 - mmcv - INFO - 
reid.backbone.layer2.1.conv3.weight - torch.Size([512, 128, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,240 - mmcv - INFO - 
reid.backbone.layer2.1.bn3.weight - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,241 - mmcv - INFO - 
reid.backbone.layer2.1.bn3.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,241 - mmcv - INFO - 
reid.backbone.layer2.2.conv1.weight - torch.Size([128, 512, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,242 - mmcv - INFO - 
reid.backbone.layer2.2.bn1.weight - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,242 

2022-11-08 15:38:39,263 - mmcv - INFO - 
reid.backbone.layer3.2.bn3.bias - torch.Size([1024]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,263 - mmcv - INFO - 
reid.backbone.layer3.3.conv1.weight - torch.Size([256, 1024, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,263 - mmcv - INFO - 
reid.backbone.layer3.3.bn1.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,264 - mmcv - INFO - 
reid.backbone.layer3.3.bn1.bias - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,264 - mmcv - INFO - 
reid.backbone.layer3.3.conv2.weight - torch.Size([256, 256, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,265 - mmcv - INFO - 
reid.backbone.layer3.3.bn2.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,26

2022-11-08 15:38:39,302 - mmcv - INFO - 
reid.backbone.layer4.2.bn1.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,302 - mmcv - INFO - 
reid.backbone.layer4.2.conv2.weight - torch.Size([512, 512, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,302 - mmcv - INFO - 
reid.backbone.layer4.2.bn2.weight - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,304 - mmcv - INFO - 
reid.backbone.layer4.2.bn2.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,305 - mmcv - INFO - 
reid.backbone.layer4.2.conv3.weight - torch.Size([2048, 512, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,305 - mmcv - INFO - 
reid.backbone.layer4.2.bn3.weight - torch.Size([2048]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:38:39,30

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 118/118, 12.6 task/s, elapsed: 9s, ETA:     0s---CLEAR MOT Evaluation---
Accumulating...
Evaluating...
Rendering...

            IDF1  MOTA  MOTP  FP  FN IDSw  Rcll  Prcn MT PT ML  FM
pedestrian 59.2% 35.7% 0.744 255 521   60 60.0% 75.4%  7  9  4  57
people     54.5% 23.2% 0.744 234 207   18 65.4% 62.6%  6  2  4  14
OVERALL    57.6% 31.8% 0.744 489 728   78 61.7% 70.5% 13 11  8  71
AVERAGE    56.9% 29.5% 0.744 489 728   78 62.7% 69.0% 13 11  8  71
Evaluation finishes with 0.45 s.
{'IDF1': 0.576, 'MOTA': 0.318, 'MOTP': 0.744, 'FP': 489, 'FN': 728, 'IDSw': 78, 'Rcll': 0.617, 'Prcn': 0.705, 'MT': 13, 'PT': 11, 'ML': 8, 'FM': 71, 'track_OVERALL_copypaste': '0.576 0.318 0.744 489 728 78 0.617 0.705 13 11 8 71 ', 'track_AVERAGE_copypaste': '0.569 0.295 0.744 489 728 78 0.627 0.690 13 11 8 71 '}


# 推理（检测+跟踪）

In [7]:
# run mot demo
import mmcv
import tempfile
from mmtrack.apis import inference_mot, init_model
# mot_config = './configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_visdrone_mot.py'
cfg = mmcv.Config.fromfile('./configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_visdrone_mot.py')
cfg.model.detector.init_cfg.checkpoint = './tutorial_exps/detector/epoch_4.pth'
cfg.model.reid.init_cfg.checkpoint = './tutorial_exps/reid/epoch_2.pth'
input_video = './demo/visdrone_demo.mp4'
imgs = mmcv.VideoReader(input_video)
# build the model from a config file
# mot_model = init_model(mot_config, device='cuda:0')
mot_model = init_model(cfg)
prog_bar = mmcv.ProgressBar(len(imgs))
out_dir = tempfile.TemporaryDirectory()
out_path = out_dir.name
# test and show/save the images
for i, img in enumerate(imgs):
    result = inference_mot(mot_model, img, frame_id=i)
    mot_model.show_result(
            img,
            result,
            show=False,
            wait_time=int(1000. / imgs.fps),
            out_file=f'{out_path}/{i:06d}.jpg')
    prog_bar.update()

output = './demo/visdrone_demo_result.mp4'
print(f'\n making the output video at {output} with a FPS of {imgs.fps}')
mmcv.frames2video(out_path, output, fps=imgs.fps, fourcc='mp4v')
out_dir.cleanup()

2022-11-08 15:39:11,780 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/detector/epoch_4.pth'}
2022-11-08 15:39:11,781 - mmcv - INFO - load model from: ./tutorial_exps/detector/epoch_4.pth
2022-11-08 15:39:11,782 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/detector/epoch_4.pth
2022-11-08 15:39:11,993 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/reid/epoch_2.pth'}
2022-11-08 15:39:11,994 - mmcv - INFO - load model from: ./tutorial_exps/reid/epoch_2.pth
2022-11-08 15:39:11,994 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/reid/epoch_2.pth
2022-11-08 15:39:12,113 - mmcv - INFO - 
detector.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,113 - mmcv - INFO - 
detector.backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps

2022-11-08 15:39:12,134 - mmcv - INFO - 
detector.backbone.layer2.0.downsample.1.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,134 - mmcv - INFO - 
detector.backbone.layer2.1.conv1.weight - torch.Size([128, 512, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,135 - mmcv - INFO - 
detector.backbone.layer2.1.bn1.weight - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,135 - mmcv - INFO - 
detector.backbone.layer2.1.bn1.bias - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,136 - mmcv - INFO - 
detector.backbone.layer2.1.conv2.weight - torch.Size([128, 128, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,136 - mmcv - INFO - 
detector.backbone.layer2.1.bn2.weight - torch.Size([128]): 
PretrainedInit: load from ./tutor

2022-11-08 15:39:12,161 - mmcv - INFO - 
detector.backbone.layer3.1.bn3.bias - torch.Size([1024]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,161 - mmcv - INFO - 
detector.backbone.layer3.2.conv1.weight - torch.Size([256, 1024, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,162 - mmcv - INFO - 
detector.backbone.layer3.2.bn1.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,162 - mmcv - INFO - 
detector.backbone.layer3.2.bn1.bias - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,162 - mmcv - INFO - 
detector.backbone.layer3.2.conv2.weight - torch.Size([256, 256, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,163 - mmcv - INFO - 
detector.backbone.layer3.2.bn2.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exp

2022-11-08 15:39:12,186 - mmcv - INFO - 
detector.backbone.layer4.0.downsample.1.bias - torch.Size([2048]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,186 - mmcv - INFO - 
detector.backbone.layer4.1.conv1.weight - torch.Size([512, 2048, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,187 - mmcv - INFO - 
detector.backbone.layer4.1.bn1.weight - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,187 - mmcv - INFO - 
detector.backbone.layer4.1.bn1.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,187 - mmcv - INFO - 
detector.backbone.layer4.1.conv2.weight - torch.Size([512, 512, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,188 - mmcv - INFO - 
detector.backbone.layer4.1.bn2.weight - torch.Size([512]): 
PretrainedInit: load from ./tut

2022-11-08 15:39:12,205 - mmcv - INFO - 
detector.roi_head.bbox_head.shared_fcs.1.bias - torch.Size([1024]): 
PretrainedInit: load from ./tutorial_exps/detector/epoch_4.pth 
 
2022-11-08 15:39:12,205 - mmcv - INFO - 
reid.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,206 - mmcv - INFO - 
reid.backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,206 - mmcv - INFO - 
reid.backbone.bn1.bias - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,206 - mmcv - INFO - 
reid.backbone.layer1.0.conv1.weight - torch.Size([64, 64, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,207 - mmcv - INFO - 
reid.backbone.layer1.0.bn1.weight - torch.Size([64]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,207 - mmcv - INFO - 

2022-11-08 15:39:12,235 - mmcv - INFO - 
reid.backbone.layer2.1.bn2.bias - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,239 - mmcv - INFO - 
reid.backbone.layer2.1.conv3.weight - torch.Size([512, 128, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,239 - mmcv - INFO - 
reid.backbone.layer2.1.bn3.weight - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,240 - mmcv - INFO - 
reid.backbone.layer2.1.bn3.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,240 - mmcv - INFO - 
reid.backbone.layer2.2.conv1.weight - torch.Size([128, 512, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,240 - mmcv - INFO - 
reid.backbone.layer2.2.bn1.weight - torch.Size([128]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,241 

2022-11-08 15:39:12,261 - mmcv - INFO - 
reid.backbone.layer3.2.bn3.bias - torch.Size([1024]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,262 - mmcv - INFO - 
reid.backbone.layer3.3.conv1.weight - torch.Size([256, 1024, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,262 - mmcv - INFO - 
reid.backbone.layer3.3.bn1.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,262 - mmcv - INFO - 
reid.backbone.layer3.3.bn1.bias - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,263 - mmcv - INFO - 
reid.backbone.layer3.3.conv2.weight - torch.Size([256, 256, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,263 - mmcv - INFO - 
reid.backbone.layer3.3.bn2.weight - torch.Size([256]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,26

2022-11-08 15:39:12,291 - mmcv - INFO - 
reid.backbone.layer4.2.bn1.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,292 - mmcv - INFO - 
reid.backbone.layer4.2.conv2.weight - torch.Size([512, 512, 3, 3]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,292 - mmcv - INFO - 
reid.backbone.layer4.2.bn2.weight - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,293 - mmcv - INFO - 
reid.backbone.layer4.2.bn2.bias - torch.Size([512]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,293 - mmcv - INFO - 
reid.backbone.layer4.2.conv3.weight - torch.Size([2048, 512, 1, 1]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,294 - mmcv - INFO - 
reid.backbone.layer4.2.bn3.weight - torch.Size([2048]): 
PretrainedInit: load from ./tutorial_exps/reid/epoch_2.pth 
 
2022-11-08 15:39:12,29

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 109/109, 9.4 task/s, elapsed: 12s, ETA:     0s
 making the output video at ./demo/visdrone_demo_result.mp4 with a FPS of 10.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 109/109, 65.5 task/s, elapsed: 2s, ETA:     0s
